In [1]:
import os
import json
import pandas as pd
import os
import numpy as np # type: ignore
from datetime import datetime
import sys

module_path = os.path.abspath(os.path.join('/Users/juanmanuelpaiba/Documents/Juan_Paiba/new_or_used_algorithm_MELI/', 'python_scripts'))
if module_path not in sys.path:
    sys.path.append(module_path)
import utilities_meli# type: ignore
#!pip install feature_engine
from feature_engine.encoding import RareLabelEncoder

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

os.getcwd()
os.chdir(path="/Users/juanmanuelpaiba/Documents/Juan_Paiba/new_or_used_algorithm_MELI")

In [3]:
# Read the Parquet files back into DataFrames
df_products = pd.read_parquet("data/Outputs/df_eda.parquet")

# Display the first few rows of the loaded DataFrames
print("Loaded Training DataFrame head:")
(df_products.head())

Loaded Training DataFrame head:


,warranty,condition,base_price,seller_id,site_id,listing_type_id,price,buying_mode,parent_item_id,category_id,last_updated,international_delivery_mode,id,official_store_id,differential_pricing,accepts_mercadopago,original_price,currency_id,title,automatic_relist,date_created,stop_time,status,video_id,catalog_product_id,subtitle,initial_quantity,start_time,permalink,sold_quantity,available_quantity,country_name,country_id,state_name,state_id,city_name,city_id,local_pick_up,free_shipping,mode,dimensions,descrip_mdo_0,id_mdo_0,type_mdo_0,season_name,gender_name,target
0,None,new,80.0,8208882349,MLA,bronze,80.0,buy_it_now,MLA6553902747,MLA126406,2015-09-05T20:42:58.000Z,none,MLA4695330653,NaN,NaN,True,NaN,ARS,Auriculares Samsung Originales Manos Libres Ca...,False,2015-09-05T20:42:53.000Z,2015-11-04 20:42:53,active,None,NaN,NaN,1,2015-09-05 20:42:53,http://articulo.mercadolibre.com.ar/MLA4695330...,0,1,Argentina,AR,Capital Federal,AR-C,San Cristóbal,TUxBQlNBTjkwNTZa,True,False,not_specified,None,Transferencia bancaria,MLATB,G,None,None,1
1,NUESTRA REPUTACION,used,2650.0,8141699488,MLA,silver,2650.0,buy_it_now,MLA7727150374,MLA10267,2015-09-26T18:08:34.000Z,none,MLA7160447179,NaN,NaN,True,NaN,ARS,Cuchillo Daga Acero Carbón Casco Yelmo Solinge...,False,2015-09-26T18:08:30.000Z,2015-11-25 18:08:30,active,None,NaN,NaN,1,2015-09-26 18:08:30,http://articulo.mercadolibre.com.ar/MLA7160447...,0,1,Argentina,AR,Capital Federal,AR-C,Buenos Aires,,True,False,me2,None,Transferencia bancaria,MLATB,G,None,None,0
2,None,used,60.0,8386096505,MLA,bronze,60.0,buy_it_now,MLA6561247998,MLA1227,2015-09-09T23:57:10.000Z,none,MLA7367189936,NaN,NaN,True,NaN,ARS,"Antigua Revista Billiken, N° 1826, Año 1954",False,2015-09-09T23:57:07.000Z,2015-11-08 23:57:07,active,None,NaN,NaN,1,2015-09-09 23:57:07,http://articulo.mercadolibre.com.ar/MLA7367189...,0,1,Argentina,AR,Capital Federal,AR-C,Boedo,TUxBQkJPRTQ0OTRa,True,False,me2,None,Transferencia bancaria,MLATB,G,None,None,0
3,None,new,580.0,5377752182,MLA,silver,580.0,buy_it_now,None,MLA86345,2015-10-05T16:03:50.306Z,none,MLA9191625553,NaN,NaN,True,NaN,ARS,Alarma Guardtex Gx412 Seguridad Para El Automo...,False,2015-09-28T18:47:56.000Z,2015-12-04 01:13:16,active,None,NaN,NaN,1,2015-09-28 18:47:56,http://articulo.mercadolibre.com.ar/MLA9191625...,0,1,Argentina,AR,Capital Federal,AR-C,Floresta,TUxBQkZMTzg5MjFa,True,False,me2,None,Transferencia bancaria,MLATB,G,None,None,1
4,MI REPUTACION.,used,30.0,2938071313,MLA,bronze,30.0,buy_it_now,MLA3133256685,MLA41287,2015-08-28T13:37:41.000Z,none,MLA7787961817,NaN,NaN,True,NaN,ARS,Serenata - Jennifer Blake,False,2015-08-24T22:07:20.000Z,2015-10-23 22:07:20,active,None,NaN,NaN,1,2015-08-24 22:07:20,http://articulo.mercadolibre.com.ar/MLA7787961...,0,1,Argentina,AR,Buenos Aires,AR-B,Tres de febrero,TUxBQ1RSRTMxODE5NA,True,False,not_specified,None,Transferencia bancaria,MLATB,G,None,None,0


In [4]:
df_products['warranty_cleaned'] = df_products['warranty'].apply(lambda x: utilities_meli.clean_warranty(x))

In [5]:
# Step 2: Apply this function to the 'title' column to create a new column 'first_two_words'
df_products['first_two_words_title'] = df_products['title'].apply(utilities_meli.extract_first_two_words)
df_products['first_word_title'] = df_products['title'].apply(utilities_meli.extract_first_word)
df_products['title_2'] = df_products['title'].apply(lambda x: utilities_meli.clean_title(x))

In [6]:
# List of rare values to replace
rare_values_to_replace = [-2147483648, 11111111, 1111111111, 8888888, 9000000, 123456789, 112111111]

# Replace rare values in 'base_price' column
df_products_00 = utilities_meli.replace_rare_values(df_products, 'base_price', rare_values_to_replace)
df_products_00 = utilities_meli.replace_rare_values(df_products_00, 'base_price', rare_values_to_replace)

In [7]:
# 'date_created' a tipo datetime
df_products_00['date_created_month'] = pd.to_datetime(df_products_00['date_created']).dt.strftime('%Y%m')

In [8]:
df_products_00['mode'].value_counts()

me2              51255
not_specified    45171
custom            3481
me1                 93
Name: mode, dtype: int64

In [9]:
df_products_00= df_products_00.drop(columns=['permalink','seller_id','warranty','condition','site_id','international_delivery_mode',
                                      'parent_item_id','last_updated','id','title','catalog_product_id',
                                      'dimensions','city_name','stop_time', 'start_time'])


In [10]:
#############################################
# Missing por cada variable
#############################################
miss_perc = (df_products_00.isnull().sum() / len(df_products_00)) * 100
print(miss_perc.sort_values(ascending=False))

differential_pricing     100.000
subtitle                 100.000
original_price            99.857
official_store_id         99.182
video_id                  97.015
gender_name               89.115
season_name               87.575
descrip_mdo_0             30.559
id_mdo_0                  30.559
type_mdo_0                30.559
local_pick_up              0.000
free_shipping              0.000
mode                       0.000
base_price                 0.000
state_id                   0.000
target                     0.000
warranty_cleaned           0.000
first_two_words_title      0.000
first_word_title           0.000
title_2                    0.000
city_id                    0.000
country_name               0.000
state_name                 0.000
country_id                 0.000
listing_type_id            0.000
available_quantity         0.000
sold_quantity              0.000
initial_quantity           0.000
status                     0.000
date_created               0.000
automatic_

In [11]:
df_products_00 = df_products_00.drop(columns=['differential_pricing','original_price',
                                              'official_store_id','date_created'])

In [12]:
# Cantidad de valores nulos en cada columna
vl_nul_column = df_products_00.isnull().sum()

# Columnas que tienen valores nulos
column_nulos = vl_nul_column[vl_nul_column > 0]
print("Variables con valores nulos:")
print(column_nulos)

Variables con valores nulos:
video_id          97015
subtitle         100000
descrip_mdo_0     30559
id_mdo_0          30559
type_mdo_0        30559
season_name       87575
gender_name       89115
dtype: int64


In [13]:
col_categoricas = df_products_00.select_dtypes('object').columns
col_categoricas

Index(['listing_type_id', 'buying_mode', 'category_id', 'currency_id',
       'status', 'video_id', 'country_name', 'country_id', 'state_name',
       'state_id', 'city_id', 'mode', 'descrip_mdo_0', 'id_mdo_0',
       'type_mdo_0', 'season_name', 'gender_name', 'warranty_cleaned',
       'first_two_words_title', 'first_word_title', 'title_2',
       'date_created_month'],
      dtype='object')

In [14]:
###################################################
# Transfromo las variables añado categoria rare
###################################################
var_dummies = ['listing_type_id', 'buying_mode', 'category_id', 'currency_id',
       'status', 'video_id', 'country_name', 'country_id', 'state_name',
       'state_id', 'city_id', 'mode', 'descrip_mdo_0', 'id_mdo_0',
       'type_mdo_0', 'season_name', 'gender_name', 'warranty_cleaned',
       'first_two_words_title', 'first_word_title', 'title_2',
       'date_created_month']

# groups rare or infrequent categories in a new category called “Rare”, or any other name entered by the user
encoder = RareLabelEncoder(tol=0.03, n_categories=2, variables=var_dummies, replace_with='Rare', missing_values='ignore')

encoder.fit(df_products_00)
# transform the data
df_products_01 = encoder.transform(df_products_00)

/Users/juanmanuelpaiba/.pyenv/versions/3.8.17/lib/python3.8/site-packages/feature_engine/encoding/rare_label.py:216: UserWarning: The number of unique categories for variable currency_id is less than that indicated in n_categories. Thus, all categories will be considered frequent
  warnings.warn(
/Users/juanmanuelpaiba/.pyenv/versions/3.8.17/lib/python3.8/site-packages/feature_engine/encoding/rare_label.py:216: UserWarning: The number of unique categories for variable country_name is less than that indicated in n_categories. Thus, all categories will be considered frequent
  warnings.warn(
/Users/juanmanuelpaiba/.pyenv/versions/3.8.17/lib/python3.8/site-packages/feature_engine/encoding/rare_label.py:216: UserWarning: The number of unique categories for variable country_id is less than that indicated in n_categories. Thus, all categories will be considered frequent
  warnings.warn(


In [15]:
df_products_01.head()

,base_price,listing_type_id,price,buying_mode,category_id,accepts_mercadopago,currency_id,automatic_relist,status,video_id,subtitle,initial_quantity,sold_quantity,available_quantity,country_name,country_id,state_name,state_id,city_id,local_pick_up,free_shipping,mode,descrip_mdo_0,id_mdo_0,type_mdo_0,season_name,gender_name,target,warranty_cleaned,first_two_words_title,first_word_title,title_2,date_created_month
0,80.0,bronze,80.0,buy_it_now,Rare,True,ARS,False,active,Rare,NaN,1,0,1,Argentina,AR,Capital Federal,AR-C,Rare,True,False,not_specified,Transferencia bancaria,MLATB,G,Rare,Rare,1,missing,Rare,Rare,otros,201509
1,2650.0,silver,2650.0,buy_it_now,Rare,True,ARS,False,active,Rare,NaN,1,0,1,Argentina,AR,Capital Federal,AR-C,,True,False,me2,Transferencia bancaria,MLATB,G,Rare,Rare,0,otros,Rare,Rare,otros,201509
2,60.0,bronze,60.0,buy_it_now,MLA1227,True,ARS,False,active,Rare,NaN,1,0,1,Argentina,AR,Capital Federal,AR-C,Rare,True,False,me2,Transferencia bancaria,MLATB,G,Rare,Rare,0,missing,Rare,Rare,otros,201509
3,580.0,silver,580.0,buy_it_now,Rare,True,ARS,False,active,Rare,NaN,1,0,1,Argentina,AR,Capital Federal,AR-C,Rare,True,False,me2,Transferencia bancaria,MLATB,G,Rare,Rare,1,missing,Rare,Rare,otros,201509
4,30.0,bronze,30.0,buy_it_now,Rare,True,ARS,False,active,Rare,NaN,1,0,1,Argentina,AR,Buenos Aires,AR-B,Rare,True,False,not_specified,Transferencia bancaria,MLATB,G,Rare,Rare,0,otros,Rare,Rare,otros,201508


In [16]:
##############################
#Validacion Manual variables
##############################
conteo_valores = df_products_01['date_created_month'].value_counts()
print(conteo_valores)

201509    48882
201510    27604
201508    22882
Rare        632
Name: date_created_month, dtype: int64


In [17]:
df_products_01.head()

,base_price,listing_type_id,price,buying_mode,category_id,accepts_mercadopago,currency_id,automatic_relist,status,video_id,subtitle,initial_quantity,sold_quantity,available_quantity,country_name,country_id,state_name,state_id,city_id,local_pick_up,free_shipping,mode,descrip_mdo_0,id_mdo_0,type_mdo_0,season_name,gender_name,target,warranty_cleaned,first_two_words_title,first_word_title,title_2,date_created_month
0,80.0,bronze,80.0,buy_it_now,Rare,True,ARS,False,active,Rare,NaN,1,0,1,Argentina,AR,Capital Federal,AR-C,Rare,True,False,not_specified,Transferencia bancaria,MLATB,G,Rare,Rare,1,missing,Rare,Rare,otros,201509
1,2650.0,silver,2650.0,buy_it_now,Rare,True,ARS,False,active,Rare,NaN,1,0,1,Argentina,AR,Capital Federal,AR-C,,True,False,me2,Transferencia bancaria,MLATB,G,Rare,Rare,0,otros,Rare,Rare,otros,201509
2,60.0,bronze,60.0,buy_it_now,MLA1227,True,ARS,False,active,Rare,NaN,1,0,1,Argentina,AR,Capital Federal,AR-C,Rare,True,False,me2,Transferencia bancaria,MLATB,G,Rare,Rare,0,missing,Rare,Rare,otros,201509
3,580.0,silver,580.0,buy_it_now,Rare,True,ARS,False,active,Rare,NaN,1,0,1,Argentina,AR,Capital Federal,AR-C,Rare,True,False,me2,Transferencia bancaria,MLATB,G,Rare,Rare,1,missing,Rare,Rare,otros,201509
4,30.0,bronze,30.0,buy_it_now,Rare,True,ARS,False,active,Rare,NaN,1,0,1,Argentina,AR,Buenos Aires,AR-B,Rare,True,False,not_specified,Transferencia bancaria,MLATB,G,Rare,Rare,0,otros,Rare,Rare,otros,201508


In [18]:
df_products_02 = utilities_meli.calculate_group_stats(df_products_01, 'mode', 'base_price')
df_products_02 = utilities_meli.calculate_group_stats(df_products_02, 'status', 'base_price')
df_products_02 = utilities_meli.calculate_group_stats(df_products_02, 'listing_type_id', 'base_price')
df_products_02 = utilities_meli.calculate_group_stats(df_products_02, 'state_id', 'base_price')
df_products_02 = utilities_meli.calculate_group_stats(df_products_02, 'automatic_relist', 'base_price')
df_products_02 = utilities_meli.calculate_group_stats(df_products_02, 'accepts_mercadopago', 'base_price')
df_products_02 = utilities_meli.calculate_group_stats(df_products_02, 'local_pick_up', 'base_price')
df_products_02 = utilities_meli.calculate_group_stats(df_products_02, 'free_shipping', 'base_price')
df_products_02 = utilities_meli.calculate_group_stats(df_products_02, 'warranty_cleaned', 'base_price')

In [19]:
df_products_02 = utilities_meli.calculate_group_stats(df_products_02, 'mode', 'initial_quantity')
df_products_02 = utilities_meli.calculate_group_stats(df_products_02, 'status', 'initial_quantity')
df_products_02 = utilities_meli.calculate_group_stats(df_products_02, 'listing_type_id', 'initial_quantity')
df_products_02 = utilities_meli.calculate_group_stats(df_products_02, 'state_id', 'initial_quantity')
df_products_02 = utilities_meli.calculate_group_stats(df_products_02, 'automatic_relist', 'initial_quantity')
df_products_02 = utilities_meli.calculate_group_stats(df_products_02, 'accepts_mercadopago', 'initial_quantity')
df_products_02 = utilities_meli.calculate_group_stats(df_products_02, 'local_pick_up', 'initial_quantity')
df_products_02 = utilities_meli.calculate_group_stats(df_products_02, 'free_shipping', 'initial_quantity')
df_products_02 = utilities_meli.calculate_group_stats(df_products_02, 'warranty_cleaned', 'initial_quantity')

In [20]:
df_products_02 = utilities_meli.calculate_group_stats(df_products_02, 'mode', 'sold_quantity')
df_products_02 = utilities_meli.calculate_group_stats(df_products_02, 'status', 'sold_quantity')
df_products_02 = utilities_meli.calculate_group_stats(df_products_02, 'listing_type_id', 'sold_quantity')
df_products_02 = utilities_meli.calculate_group_stats(df_products_02, 'state_id', 'sold_quantity')
df_products_02 = utilities_meli.calculate_group_stats(df_products_02, 'automatic_relist', 'sold_quantity')
df_products_02 = utilities_meli.calculate_group_stats(df_products_02, 'accepts_mercadopago', 'sold_quantity')
df_products_02 = utilities_meli.calculate_group_stats(df_products_02, 'local_pick_up', 'sold_quantity')
df_products_02 = utilities_meli.calculate_group_stats(df_products_02, 'free_shipping', 'sold_quantity')
df_products_02 = utilities_meli.calculate_group_stats(df_products_02, 'warranty_cleaned', 'sold_quantity')

In [21]:
###########################################
## Paso Variables Categoricas a Dummies
###########################################
df_products_02 = pd.get_dummies(df_products_02)
# Reemplazar NaN con 0
df_products_02 = df_products_02.fillna(0)
# Convertir True y False a 1 y 0
df_products_02 = df_products_02.astype(int)

In [22]:
df_products_02.head()

,base_price,price,accepts_mercadopago,automatic_relist,subtitle,initial_quantity,sold_quantity,available_quantity,local_pick_up,free_shipping,target,mean_base_price_mode,min_base_price_mode,max_base_price_mode,var_base_price_mode,median_base_price_mode,std_base_price_mode,q1_base_price_mode,q3_base_price_mode,mean_base_price_status,min_base_price_status,max_base_price_status,var_base_price_status,median_base_price_status,std_base_price_status,q1_base_price_status,q3_base_price_status,mean_base_price_listing_type_id,min_base_price_listing_type_id,max_base_price_listing_type_id,var_base_price_listing_type_id,median_base_price_listing_type_id,std_base_price_listing_type_id,q1_base_price_listing_type_id,q3_base_price_listing_type_id,mean_base_price_state_id,min_base_price_state_id,max_base_price_state_id,var_base_price_state_id,median_base_price_state_id,std_base_price_state_id,q1_base_price_state_id,q3_base_price_state_id,mean_base_price_automatic_relist,min_base_price_automatic_relist,max_base_price_automatic_relist,var_base_price_automatic_relist,median_base_price_automatic_relist,std_base_price_automatic_relist,q1_base_price_automatic_relist,q3_base_price_automatic_relist,mean_base_price_accepts_mercadopago,min_base_price_accepts_mercadopago,max_base_price_accepts_mercadopago,var_base_price_accepts_mercadopago,median_base_price_accepts_mercadopago,std_base_price_accepts_mercadopago,q1_base_price_accepts_mercadopago,q3_base_price_accepts_mercadopago,mean_base_price_local_pick_up,min_base_price_local_pick_up,max_base_price_local_pick_up,var_base_price_local_pick_up,median_base_price_local_pick_up,std_base_price_local_pick_up,q1_base_price_local_pick_up,q3_base_price_local_pick_up,mean_base_price_free_shipping,min_base_price_free_shipping,max_base_price_free_shipping,var_base_price_free_shipping,median_base_price_free_shipping,std_base_price_free_shipping,q1_base_price_free_shipping,q3_base_price_free_shipping,mean_base_price_warranty_cleaned,min_base_price_warranty_cleaned,max_base_price_warranty_cleaned,var_base_price_warranty_cleaned,median_base_price_warranty_cleaned,std_base_price_warranty_cleaned,q1_base_price_warranty_cleaned,q3_base_price_warranty_cleaned,mean_initial_quantity_mode,min_initial_quantity_mode,max_initial_quantity_mode,var_initial_quantity_mode,median_initial_quantity_mode,std_initial_quantity_mode,q1_initial_quantity_mode,q3_initial_quantity_mode,mean_initial_quantity_status,min_initial_quantity_status,max_initial_quantity_status,var_initial_quantity_status,median_initial_quantity_status,std_initial_quantity_status,q1_initial_quantity_status,q3_initial_quantity_status,mean_initial_quantity_listing_type_id,min_initial_quantity_listing_type_id,max_initial_quantity_listing_type_id,var_initial_quantity_listing_type_id,median_initial_quantity_listing_type_id,std_initial_quantity_listing_type_id,q1_initial_quantity_listing_type_id,q3_initial_quantity_listing_type_id,mean_initial_quantity_state_id,min_initial_quantity_state_id,max_initial_quantity_state_id,var_initial_quantity_state_id,median_initial_quantity_state_id,std_initial_quantity_state_id,q1_initial_quantity_state_id,q3_initial_quantity_state_id,mean_initial_quantity_automatic_relist,min_initial_quantity_automatic_relist,max_initial_quantity_automatic_relist,var_initial_quantity_automatic_relist,median_initial_quantity_automatic_relist,std_initial_quantity_automatic_relist,q1_initial_quantity_automatic_relist,q3_initial_quantity_automatic_relist,mean_initial_quantity_accepts_mercadopago,min_initial_quantity_accepts_mercadopago,max_initial_quantity_accepts_mercadopago,var_initial_quantity_accepts_mercadopago,median_initial_quantity_accepts_mercadopago,std_initial_quantity_accepts_mercadopago,q1_initial_quantity_accepts_mercadopago,q3_initial_quantity_accepts_mercadopago,mean_initial_quantity_local_pick_up,min_initial_quantity_local_pick_up,max_initial_quantity_local_pick_up,var_initial_quantity_local_pick_up,median_initial_quantity_local_pick_up,std_initia

In [23]:
df_products_02 = utilities_meli.create_transformed_columns(df_products_02,'price')
df_products_02 = utilities_meli.create_transformed_columns(df_products_02,'initial_quantity')
df_products_02 = utilities_meli.create_transformed_columns(df_products_02,'sold_quantity')

/Users/juanmanuelpaiba/.pyenv/versions/3.8.17/lib/python3.8/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in arcsin
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/juanmanuelpaiba/.pyenv/versions/3.8.17/lib/python3.8/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in arcsin
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/juanmanuelpaiba/.pyenv/versions/3.8.17/lib/python3.8/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in arcsin
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/juanmanuelpaiba/.pyenv/versions/3.8.17/lib/python3.8/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in arcsin
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/juanmanuelpaiba/.pyenv/versions/3.8.17/lib/python3.8/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in arcsin
  result

In [24]:
df_products_02['price_x_initial_quantity'] = df_products_02['price']*df_products_02['initial_quantity']
df_products_02['price_x_sold_quantity'] = df_products_02['price']*df_products_02['sold_quantity']
df_products_02['price_2_x_sold_quantity'] = df_products_02['price_square']*df_products_02['sold_quantity']

In [25]:
df_products_02.to_parquet("data/Outputs/df_modelling.parquet")

In [26]:
df_products_02.head()

,base_price,price,accepts_mercadopago,automatic_relist,subtitle,initial_quantity,sold_quantity,available_quantity,local_pick_up,free_shipping,target,mean_base_price_mode,min_base_price_mode,max_base_price_mode,var_base_price_mode,median_base_price_mode,std_base_price_mode,q1_base_price_mode,q3_base_price_mode,mean_base_price_status,min_base_price_status,max_base_price_status,var_base_price_status,median_base_price_status,std_base_price_status,q1_base_price_status,q3_base_price_status,mean_base_price_listing_type_id,min_base_price_listing_type_id,max_base_price_listing_type_id,var_base_price_listing_type_id,median_base_price_listing_type_id,std_base_price_listing_type_id,q1_base_price_listing_type_id,q3_base_price_listing_type_id,mean_base_price_state_id,min_base_price_state_id,max_base_price_state_id,var_base_price_state_id,median_base_price_state_id,std_base_price_state_id,q1_base_price_state_id,q3_base_price_state_id,mean_base_price_automatic_relist,min_base_price_automatic_relist,max_base_price_automatic_relist,var_base_price_automatic_relist,median_base_price_automatic_relist,std_base_price_automatic_relist,q1_base_price_automatic_relist,q3_base_price_automatic_relist,mean_base_price_accepts_mercadopago,min_base_price_accepts_mercadopago,max_base_price_accepts_mercadopago,var_base_price_accepts_mercadopago,median_base_price_accepts_mercadopago,std_base_price_accepts_mercadopago,q1_base_price_accepts_mercadopago,q3_base_price_accepts_mercadopago,mean_base_price_local_pick_up,min_base_price_local_pick_up,max_base_price_local_pick_up,var_base_price_local_pick_up,median_base_price_local_pick_up,std_base_price_local_pick_up,q1_base_price_local_pick_up,q3_base_price_local_pick_up,mean_base_price_free_shipping,min_base_price_free_shipping,max_base_price_free_shipping,var_base_price_free_shipping,median_base_price_free_shipping,std_base_price_free_shipping,q1_base_price_free_shipping,q3_base_price_free_shipping,mean_base_price_warranty_cleaned,min_base_price_warranty_cleaned,max_base_price_warranty_cleaned,var_base_price_warranty_cleaned,median_base_price_warranty_cleaned,std_base_price_warranty_cleaned,q1_base_price_warranty_cleaned,q3_base_price_warranty_cleaned,mean_initial_quantity_mode,min_initial_quantity_mode,max_initial_quantity_mode,var_initial_quantity_mode,median_initial_quantity_mode,std_initial_quantity_mode,q1_initial_quantity_mode,q3_initial_quantity_mode,mean_initial_quantity_status,min_initial_quantity_status,max_initial_quantity_status,var_initial_quantity_status,median_initial_quantity_status,std_initial_quantity_status,q1_initial_quantity_status,q3_initial_quantity_status,mean_initial_quantity_listing_type_id,min_initial_quantity_listing_type_id,max_initial_quantity_listing_type_id,var_initial_quantity_listing_type_id,median_initial_quantity_listing_type_id,std_initial_quantity_listing_type_id,q1_initial_quantity_listing_type_id,q3_initial_quantity_listing_type_id,mean_initial_quantity_state_id,min_initial_quantity_state_id,max_initial_quantity_state_id,var_initial_quantity_state_id,median_initial_quantity_state_id,std_initial_quantity_state_id,q1_initial_quantity_state_id,q3_initial_quantity_state_id,mean_initial_quantity_automatic_relist,min_initial_quantity_automatic_relist,max_initial_quantity_automatic_relist,var_initial_quantity_automatic_relist,median_initial_quantity_automatic_relist,std_initial_quantity_automatic_relist,q1_initial_quantity_automatic_relist,q3_initial_quantity_automatic_relist,mean_initial_quantity_accepts_mercadopago,min_initial_quantity_accepts_mercadopago,max_initial_quantity_accepts_mercadopago,var_initial_quantity_accepts_mercadopago,median_initial_quantity_accepts_mercadopago,std_initial_quantity_accepts_mercadopago,q1_initial_quantity_accepts_mercadopago,q3_initial_quantity_accepts_mercadopago,mean_initial_quantity_local_pick_up,min_initial_quantity_local_pick_up,max_initial_quantity_local_pick_up,var_initial_quantity_local_pick_up,median_initial_quantity_local_pick_up,std_initia